### Import RSE Net

In [1]:
import rse_net  # rse_net.py

C:\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Load Data Set

In [2]:
import numpy as np
from keras.models import Sequential
from keras.models import Model, Input
from keras.layers import Dense, Activation, Dropout, BatchNormalization, Average
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping

In [3]:
X = np.load('features_AC.npy')
y = np.load('y_response.npy')
print(X.shape)
print(y.shape)

(450, 30)
(450,)


In [4]:
from sklearn.preprocessing import LabelBinarizer 
label_binarizer = LabelBinarizer()
label_binarizer.fit([1,2,3]) # need to be global or remembered to use it later

def one_hot_encode(x):
    return label_binarizer.transform(x)

In [5]:
y_ohe = one_hot_encode(y)
print(y_ohe.shape)

(450, 3)


### Features in Random Subspace

In [6]:
rsf = rse_net.rs_features(X, n_classifier = 7, f_lb = 20, f_ub = 30) # . for reminder

In [9]:
for i in range(len(rsf)):
    print(rsf[i].shape)

(450, 30)
(450, 21)
(450, 25)
(450, 29)
(450, 23)
(450, 26)
(450, 22)


### Ensemble Network

In [10]:
models = []
input_layers = []
for i in range(len(rsf)):
    md_n, il_n = rse_net.classifier(rsf[i].shape[1]) # . for reminder
    models.append(md_n)
    input_layers.append(il_n)

In [11]:
model_rse = rse_net.ensemble(models, input_layers) # . for reminder

In [12]:
model_rse.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 30)           0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 21)           0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 25)           0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 29)           0                                            
__________________________________________________________________________________________________
input_5 (I

In [13]:
from keras.optimizers import Adam
adam = Adam(lr=0.02, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
# kept default params

In [59]:
model_rse.compile(optimizer=adam,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

### Train Ensemble Model

In [60]:
mhist = model_rse.fit(rsf, y_ohe, batch_size = 512, validation_split = 0.1,
             epochs=100, verbose = 0)

In [61]:
np.max(mhist.history['val_acc'])

0.8222222328186035

### 10-fold CV

In [10]:
from rse_net import cross_validation
v_acc_a = rse_net.cross_validation(rsf, y_ohe, k = 10)

In [11]:
print(v_acc_a)

[0.9333333373069763, 0.8666666746139526, 0.9111111164093018, 0.8444444537162781, 0.8222222328186035, 0.7555555701255798, 0.8888888955116272, 0.8666666746139526, 0.8666666746139526, 0.8444444537162781]


In [13]:
print('Accuracy: ', np.mean(v_acc_a), ' +/- ', 2*np.std(v_acc_a))

Accuracy:  0.8600000083446503  +/-  0.09333332777023315
